In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "crusher"

In [ ]:
name_re = re.compile("(weak|strong)_scaling-(\w+)-(\d+pt)-n_(\d+)-([\w_]+)_comm-(gpu|cpu)-nodes_(\d+)-ppn_(\d+)")

In [ ]:
time_re = re.compile("DURATION:\s*(\d+(\.\d+)?(e[+-]\d+)?)")

In [ ]:
size_re = re.compile("SIZE:\s*(\d+)")

In [ ]:
stencil_type = dict([((True, 2), "5pt"), ((True, 3), "7pt"), ((False, 2), "9pt"), ((False, 3), "27pt")])

In [ ]:
num_gpus = 8

In [ ]:
data = []
for file in Path(f"{root_path}/data_solvers").iterdir():
    if m := name_re.search(file.name):
        with open(file, "r") as input:
            input_str = input.read()
            duration = 0
            size = 0
            if t_m := time_re.search(input_str):
                duration = t_m.group(1)
            if s_m := size_re.search(input_str):
                size = s_m.group(1)
            data.append({"scaling": m.group(1), "solver": m.group(2), "stencil": m.group(3), "n": m.group(4),
                         "comm_pattern": m.group(5),
                         "type": m.group(6), "nodes": m.group(7), "ppn": m.group(8), "time": duration, "size": size})
df = pd.DataFrame(data).drop(columns=["scaling", "type"]).astype({"n": int, "nodes": int, "ppn": int, "time": float, "size": int})
df

In [ ]:
df["gpus"] = df["nodes"] * df["ppn"]

In [ ]:
piv = df.drop(columns=["n", "size", "nodes", "stencil", "comm_pattern"]).pivot(index="gpus", columns=["solver"], values="time")
piv = piv[piv.columns.sort_values()]
piv

In [ ]:
df[["n", "stencil", "solver"]].agg(lambda x: x[2], axis=1)

In [ ]:
df["flop"] = df[["size", "stencil", "solver"]].agg(lambda x: flops[x[2]](x[0], x[0] * int(x[1][:-2])), axis=1)
df["flop/s"] = df.flop / df.time
df

In [ ]:
def bicgstab(n, nnz):
    return 2 * n + 8 * n + 2 * nnz + 2 * n + 3 * n + 2 * nnz + 2 * n + 2 * n + 7 * n
def cgs(n, nnz):
    return 2 * n + 7 * n + 2 * nnz + 4 * n + 2 * nnz + 4 * n
def fcg(n, nnz):
    return 2 * n + 2 * n + 3 * n + 2 * nnz + 2 * n + 6 * n
def cg(n, nnz):
    return 12 * n + 2 * nnz

flops = {"bicgstab": bicgstab,
         "cgs": cgs,
         "fcg": fcg,
         "cg": cg}

In [ ]:
piv_flops = df[df.n >= 100000].drop(columns=["size", "nodes", "flop", "time", "comm_pattern", "stencil"]).pivot(index="gpus", columns=["solver"], values="flop/s")
piv_flops = piv_flops[piv_flops.columns.sort_values()]
piv_flops

In [ ]:
_df = piv_flops
linear_scaling_x = [0] + list(_df.index) + [2000]
linear_scaling = [_df["bicgstab"][1] * 1 / 1e12 * i / _df.index[0] for i in linear_scaling_x]
linear_scaling

In [ ]:
linear_scaling_x

In [ ]:
_df = piv_flops
fig, ax = plt.subplots(dpi=300)
ax.set_prop_cycle(default_cycler)
ax.set_title(f"CG Performance per Iteration")
(_df / 1e12).plot(ax=ax, legend=True, logx=True, logy=True)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.plot(linear_scaling_x, linear_scaling, '--', color="grey", alpha=0.5)
ax.legend([n.capitalize() for n in piv_flops.columns] + ["Ideal"])
ax.set_xlim([xlim[0] * 0.85, xlim[1] * 1.15])
ax.set_ylim([ylim[0] * 0.85, ylim[1] * 1.15])
ax.set_xlabel("# GPUs")
ax.set_ylabel("TFLOP/s")
fig.savefig(f"{root_path}/img/optimal-flops.png")

In [ ]:
"3pt"[:-2]